In [1]:
import asyncio
from itertools import chain

from nbiatoolkit.logging_config import logger, console
from nbiatoolkit.nbia import NBIA_ENDPOINT, NBIAClient, get_single_image_per_series
from nbiatoolkit.settings import Settings
import pandas as pd
print = console.print

In [2]:
# Expects a .env file in the same directory with the following (double underscore is a separator):
# LOGIN__NBIA_USERNAME="..."
# LOGIN__NBIA_PASSWORD="..."

settings = Settings()
client = NBIAClient(username=settings.NBIA_USERNAME, password=settings.NBIA_PASSWORD, log_level="DEBUG")

In [3]:
collections = await client._getCollections()
print(f"Found {len(collections)=} collections")

Output()

Found len(collections)=138 collections

In [4]:
collection_series = []
for i in range(0, len(collections), 10):
    logger.info(f"Getting series for collections {i} to {i+10} of {len(collections)}")
    tasks = [client._getSeries(collection) for collection in collections[i:i+10]]
    series = await asyncio.gather(*tasks)
    collection_series.extend(list(chain(*series)))


Querying v2/getSeries... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% ⠋ Time elapsed: 0:00:06

In [5]:
collection_series
print(f'Found {len(collection_series)=} series')

Found len(collection_series)=308946 series

In [6]:
sdf = pd.DataFrame(collection_series)
sdf.to_csv('allseries.tsv', index=False, sep='\t')

In [7]:
sdf.shape

(308946, 24)